# Real-World Evidence: Identifying an approximate anti-IL-12/23 cohort in the UCH Data Warehouse

## Subpopulation1: All patients with Crohn's disease

In [ ]:
%sql
-- patients with Crohn's Disease
DROP TABLE IF EXISTS cd_cohort; 

CREATE TABLE cd_cohort (person_id INT);

INSERT INTO cd_cohort
  SELECT DISTINCT person_id 
  FROM omop_deid.concept_ancestor c1
  INNER JOIN omop_deid.condition_occurrence dx
      ON ( c1.descendant_concept_id = dx.condition_concept_id )
  WHERE c1.ancestor_concept_id IN (201606) -- CD
;

In [ ]:
sql 
-- patient metadata (age, gender, race, ethnicity)
DROP TABLE IF EXISTS cohort_metadata; 

CREATE TABLE cohort_metadata (person_id INT, current_age INT, gender STRING, race STRING, ethnicity STRING);

INSERT INTO cohort_metadata 
  SELECT  person.person_id, 
          2023 - person.year_of_birth AS current_age,
          c1.concept_name AS gender, 
          c2.concept_name AS race, 
          c3.concept_name AS ethnicity
  FROM omop_deid.person 
  INNER JOIN cd_cohort ON vr_cd.person_id = person.person_id 
  INNER JOIN omop_deid.concept c1 ON person.gender_concept_id = c1.concept_id
  INNER JOIN omop_deid.concept c2 ON person.race_concept_id = c2.concept_id
  INNER JOIN omop_deid.concept c3 ON person.ethnicity_concept_id = c3.concept_id
;

In [ ]:
%python
# convert sql result to pandas data frame
import numpy as np
import pandas as pd

results_df  = spark.sql("select * from cohort_metadata")
results_pdf = results_df.select("*").toPandas()
results_pdf.head()

In [ ]:
%python 
# map patient current age to age category
labels = ['Under 5','5 to 9',
          '10 to 14','15 to 19',
          '20 to 24','25 to 29',
          '30 to 34','35 to 39',
          '40 to 44','45 to 49',
          '50 to 54','55 to 59',
          '60 to 64','65 to 69',
          '70 to 74','75 to 79',
          '80 to 84','85 and over']

results_pdf['curr_age_cat'] = np.nan

results_pdf['curr_age_cat'] = results_pdf.apply(lambda x: labels[0]  if (x.current_age < 5) else x.curr_age_cat, axis=1)
results_pdf['curr_age_cat'] = results_pdf.apply(lambda x: labels[1]  if (x.current_age > 4) & (x.current_age < 10) else x.curr_age_cat, axis=1)
results_pdf['curr_age_cat'] = results_pdf.apply(lambda x: labels[2]  if (x.current_age > 9) & (x.current_age < 15) else x.curr_age_cat, axis=1)
results_pdf['curr_age_cat'] = results_pdf.apply(lambda x: labels[3]  if (x.current_age > 14) & (x.current_age < 20) else x.curr_age_cat, axis=1)
results_pdf['curr_age_cat'] = results_pdf.apply(lambda x: labels[4]  if (x.current_age > 19) & (x.current_age < 25) else x.curr_age_cat, axis=1)
results_pdf['curr_age_cat'] = results_pdf.apply(lambda x: labels[5]  if (x.current_age > 24) & (x.current_age < 30) else x.curr_age_cat, axis=1)
results_pdf['curr_age_cat'] = results_pdf.apply(lambda x: labels[6]  if (x.current_age > 29) & (x.current_age < 35) else x.curr_age_cat, axis=1)
results_pdf['curr_age_cat'] = results_pdf.apply(lambda x: labels[7]  if (x.current_age > 34) & (x.current_age < 40) else x.curr_age_cat, axis=1)
results_pdf['curr_age_cat'] = results_pdf.apply(lambda x: labels[8]  if (x.current_age > 39) & (x.current_age < 45) else x.curr_age_cat, axis=1)
results_pdf['curr_age_cat'] = results_pdf.apply(lambda x: labels[9]  if (x.current_age > 44) & (x.current_age < 50) else x.curr_age_cat, axis=1)
results_pdf['curr_age_cat'] = results_pdf.apply(lambda x: labels[10] if (x.current_age > 49) & (x.current_age < 55) else x.curr_age_cat, axis=1)
results_pdf['curr_age_cat'] = results_pdf.apply(lambda x: labels[11] if (x.current_age > 54) & (x.current_age < 60) else x.curr_age_cat, axis=1)
results_pdf['curr_age_cat'] = results_pdf.apply(lambda x: labels[12] if (x.current_age > 59) & (x.current_age < 65) else x.curr_age_cat, axis=1)
results_pdf['curr_age_cat'] = results_pdf.apply(lambda x: labels[13] if (x.current_age > 64) & (x.current_age < 70) else x.curr_age_cat, axis=1)
results_pdf['curr_age_cat'] = results_pdf.apply(lambda x: labels[14] if (x.current_age > 69) & (x.current_age < 75) else x.curr_age_cat, axis=1)
results_pdf['curr_age_cat'] = results_pdf.apply(lambda x: labels[15] if (x.current_age > 74) & (x.current_age < 80) else x.curr_age_cat, axis=1)
results_pdf['curr_age_cat'] = results_pdf.apply(lambda x: labels[16] if (x.current_age > 79) & (x.current_age < 85) else x.curr_age_cat, axis=1)
results_pdf['curr_age_cat'] = results_pdf.apply(lambda x: labels[17] if (x.current_age > 84) else x.curr_age_cat, axis=1)

results_pdf[['current_age','curr_age_cat']].head()

In [ ]:
%python
# print age vs gender table
cd_age_vs_gender = results_pdf[['curr_age_cat','gender','person_id']][results_pdf.gender.isin(['MALE','FEMALE'])]\
  .groupby(['curr_age_cat','gender'])\
  .count()\
  .reset_index()

pd.pivot_table(cd_age_vs_gender, values='person_id', index='curr_age_cat', columns='gender', aggfunc='first')

# 	            FEMALE	MALE
# Under 5	    7	    5
# 5 to 9	    42	    44
# 10 to 14	    112	    170
# 15 to 19	    296	    347
# 20 to 24	    568	    684
# 25 to 29	    940	    944
# 30 to 34	    1,231	1,219
# 35 to 39	    1,299	1,112
# 40 to 44	    1,156	1,045
# 45 to 49	    918	    847
# 50 to 54	    961	    816
# 55 to 59	    850	    735
# 60 to 64	    945	    791
# 65 to 69	    799	    717
# 70 to 74	    736	    658
# 75 to 79	    620	    552
# 80 to 84	    350	    332
# 85 and over	386	    293

## Subpopulation 2: All Crohn's patients who received a biologic

In [ ]:
%sql
-- same query as above, but saving to temporary table
DROP TABLE IF EXISTS taken_biologic; 

CREATE TABLE taken_biologic (person_id INT, current_age INT, gender STRING, race STRING, ethnicity STRING, medication STRING, drug_class STRING, first_start_date DATE, year INT);

INSERT INTO taken_biologic 
  WITH medications AS (
  SELECT c_des.concept_id,
      c_des.concept_name,
      c_des.vocabulary_id,
      c_des.concept_code
  FROM omop_deid.concept_ancestor des
  INNER JOIN omop_deid.concept c_des
      ON ( des.descendant_concept_id = c_des.concept_id )
  where ancestor_concept_id IN (1119119, 40161532, 937368, 735843, 45774639, 912263) -- ADA, UST, IFX, NTZ, VDZ, CZP
  ),
  biologics AS (
    SELECT  cd.*,
            rx.drug_concept_id, rx.drug_exposure_start_date,
            medications.concept_id, medications.concept_name, medications.concept_code,
            CASE 
                WHEN concept_name LIKE '%adalimumab%' THEN 'adalimumab'
                WHEN concept_name LIKE '%ustekinumab%' THEN 'ustekinumab'
                WHEN concept_name LIKE '%infliximab%' THEN 'infliximab'
                WHEN concept_name LIKE '%vedolizumab%' THEN 'vedolizumab'
                WHEN concept_name LIKE '%natalizumab%' THEN 'natalizumab'
                WHEN concept_name LIKE '%certolizumab%' THEN 'certolizumab'
                ELSE '-1'
            END AS medication,
            CASE 
                WHEN concept_name LIKE '%adalimumab%' THEN 'TNF'
                WHEN concept_name LIKE '%ustekinumab%' THEN 'IL12'
                WHEN concept_name LIKE '%infliximab%' THEN 'TNF'
                WHEN concept_name LIKE '%vedolizumab%' THEN 'Integrin'
                WHEN concept_name LIKE '%natalizumab%' THEN 'Integrin'
                WHEN concept_name LIKE '%certolizumab%' THEN 'TNF'
                ELSE '-1'
            END AS drug_class
    FROM omop_deid.drug_exposure rx 
    INNER JOIN cohort_metadata cd
      ON ( cd.person_id = rx.person_id )
    INNER JOIN medications
      ON ( medications.concept_id = rx.drug_concept_id )
    WHERE rx.person_id IS NOT NULL
  ),
  temp AS (
      -- collapse to first data patient is exposed to each biologic
      SELECT person_id, current_age, gender, race, ethnicity, medication, drug_class, MIN(drug_exposure_start_date) AS first_start_date
      FROM biologics
      GROUP BY person_id, current_age, gender, race, ethnicity, medication, drug_class
  )
    
  SELECT temp.*, YEAR(first_start_date) As year
  FROM temp
  ORDER BY person_id, first_start_date ASC
;


In [ ]:
%python
# analyze data in python
biologics_df  = spark.sql("select * from taking_biologics")
biologics_pdf = biologics_df.select("*").toPandas()
biologics_pdf.head()

In [ ]:
%python
# order medication by start date, person_id
biologics_pdf['rn'] = biologics_pdf.groupby(['person_id'])['first_start_date'].rank(method="first", ascending=True)
biologics_pdf.head(6)

In [ ]:
%python 
# map patient current age to age category
labels = ['Under 5','5 to 9',
          '10 to 14','15 to 19',
          '20 to 24','25 to 29',
          '30 to 34','35 to 39',
          '40 to 44','45 to 49',
          '50 to 54','55 to 59',
          '60 to 64','65 to 69',
          '70 to 74','75 to 79',
          '80 to 84','85 and over']

biologics_pdf['curr_age_cat'] = np.nan

biologics_pdf['curr_age_cat'] = biologics_pdf.apply(lambda x: labels[0]  if (x.current_age < 5) else x.curr_age_cat, axis=1)
biologics_pdf['curr_age_cat'] = biologics_pdf.apply(lambda x: labels[1]  if (x.current_age > 4) & (x.current_age < 10) else x.curr_age_cat, axis=1)
biologics_pdf['curr_age_cat'] = biologics_pdf.apply(lambda x: labels[2]  if (x.current_age > 9) & (x.current_age < 15) else x.curr_age_cat, axis=1)
biologics_pdf['curr_age_cat'] = biologics_pdf.apply(lambda x: labels[3]  if (x.current_age > 14) & (x.current_age < 20) else x.curr_age_cat, axis=1)
biologics_pdf['curr_age_cat'] = biologics_pdf.apply(lambda x: labels[4]  if (x.current_age > 19) & (x.current_age < 25) else x.curr_age_cat, axis=1)
biologics_pdf['curr_age_cat'] = biologics_pdf.apply(lambda x: labels[5]  if (x.current_age > 24) & (x.current_age < 30) else x.curr_age_cat, axis=1)
biologics_pdf['curr_age_cat'] = biologics_pdf.apply(lambda x: labels[6]  if (x.current_age > 29) & (x.current_age < 35) else x.curr_age_cat, axis=1)
biologics_pdf['curr_age_cat'] = biologics_pdf.apply(lambda x: labels[7]  if (x.current_age > 34) & (x.current_age < 40) else x.curr_age_cat, axis=1)
biologics_pdf['curr_age_cat'] = biologics_pdf.apply(lambda x: labels[8]  if (x.current_age > 39) & (x.current_age < 45) else x.curr_age_cat, axis=1)
biologics_pdf['curr_age_cat'] = biologics_pdf.apply(lambda x: labels[9]  if (x.current_age > 44) & (x.current_age < 50) else x.curr_age_cat, axis=1)
biologics_pdf['curr_age_cat'] = biologics_pdf.apply(lambda x: labels[10] if (x.current_age > 49) & (x.current_age < 55) else x.curr_age_cat, axis=1)
biologics_pdf['curr_age_cat'] = biologics_pdf.apply(lambda x: labels[11] if (x.current_age > 54) & (x.current_age < 60) else x.curr_age_cat, axis=1)
biologics_pdf['curr_age_cat'] = biologics_pdf.apply(lambda x: labels[12] if (x.current_age > 59) & (x.current_age < 65) else x.curr_age_cat, axis=1)
biologics_pdf['curr_age_cat'] = biologics_pdf.apply(lambda x: labels[13] if (x.current_age > 64) & (x.current_age < 70) else x.curr_age_cat, axis=1)
biologics_pdf['curr_age_cat'] = biologics_pdf.apply(lambda x: labels[14] if (x.current_age > 69) & (x.current_age < 75) else x.curr_age_cat, axis=1)
biologics_pdf['curr_age_cat'] = biologics_pdf.apply(lambda x: labels[15] if (x.current_age > 74) & (x.current_age < 80) else x.curr_age_cat, axis=1)
biologics_pdf['curr_age_cat'] = biologics_pdf.apply(lambda x: labels[16] if (x.current_age > 79) & (x.current_age < 85) else x.curr_age_cat, axis=1)
biologics_pdf['curr_age_cat'] = biologics_pdf.apply(lambda x: labels[17] if (x.current_age > 84) else x.curr_age_cat, axis=1)

biologics_pdf[['current_age','curr_age_cat']].head(

In [ ]:
%python
# print age vs gender table
biologics_age_vs_gender = biologics_pdf[['curr_age_cat','gender','person_id']][biologics_pdf.gender.isin(['MALE','FEMALE'])]\
  .drop_duplicates()\
  .groupby(['curr_age_cat','gender'])\
  .count()\
  .reset_index()

pd.pivot_table(biologics_age_vs_gender, values='person_id', index='curr_age_cat', columns='gender', aggfunc='first')

# 	            FEMALE	MALE
# Under 5	      1	    1
# 5 to 9	      14	    14
# 10 to 14	    54	    85
# 15 to 19	    159	    197
# 20 to 24	    315	    391
# 25 to 29	    507	    558
# 30 to 34	    641	    679
# 35 to 39	    616	    553
# 40 to 44	    514	    505
# 45 to 49	    363	    403
# 50 to 54	    383	    363
# 55 to 59	    319	    283
# 60 to 64	    331	    263
# 65 to 69	    252	    261
# 70 to 74	    231	    212
# 75 to 79	    170	    148
# 80 to 84	    86	    67
# 85 and over	  47	    43

## Subpopulation 3: All Crohn's patients who received their first biologic after 2016

%python
# count number of patients who started their first biologic after 2016 
# 2016 = year ustekinumab was approved by FDA for Crohn's disease
cohort = pd.DataFrame(biologics_pdf[\
            (biologics_pdf.rn == 1) &\
            (biologics_pdf.year >= 2016)
           ]['person_id'].unique(), columns = ['person_id'])

biologics_pdf2 = biologics_pdf.merge(cohort, on='person_id', how = 'inner')
print(biologics_pdf2.shape)
biologics_pdf2.head()

In [ ]:
%python
# print age vs gender table
diag_biologics = biologics_pdf2[['curr_age_cat','gender','person_id']][biologics_pdf2.gender.isin(['MALE','FEMALE'])]\
  .drop_duplicates()\
  .groupby(['curr_age_cat','gender'])\
  .count()\
  .reset_index()

pd.pivot_table(diag_biologics, values='person_id', index='curr_age_cat', columns='gender', aggfunc='first')

# 	            FEMALE	MALE
# Under 5	      1	    1
# 5 to 9	      14	    14
# 10 to 14	    54	    84
# 15 to 19	    152	    189
# 20 to 24	    272	    334
# 25 to 29	    365	    407
# 30 to 34	    457	    456
# 35 to 39	    397	    355
# 40 to 44	    296	    319
# 45 to 49	    198	    235
# 50 to 54	    227	    216
# 55 to 59	    201	    191
# 60 to 64	    213	    176
# 65 to 69	    163	    168
# 70 to 74	    161	    131
# 75 to 79	    110	    106
# 80 to 84	    48	    44
# 85 and over	  27	    28